#### If using AMD GPU, switch backend to PlaidML library:

In [1]:
import os
os.environ['KERAS_BACKEND']='plaidml.keras.backend'

In [2]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import time
import pickle
from keras.models import load_model

Using plaidml.keras.backend backend.


#### Define Experiments

In [3]:
conv_layers = [5]      # number of conv layers
layer_sizes = [32]     # number of nodes in a layer
dense_layers = [2]     # number of dense layers

#### Load input data

In [4]:
pickle_in = open('../Dataset/df_all.pickle', 'rb')
df_train, df_test = pickle.load(pickle_in)

In [5]:
# The Keras ImageDataGenerator uses string type data label
df_train['gender'] = df_train.gender.astype(str)
df_test['gender'] = df_test.gender.astype(str)

In [6]:
print(df_train.shape, df_test.shape)

(196464, 10) (10340, 10)


In [7]:
df_train.dtypes

path                object
id                  uint16
name                object
dob         datetime64[ns]
gender              object
score1             float64
score2             float64
pic_date    datetime64[ns]
region              object
age                float64
dtype: object

We will be using a generator to feed model with images, the X would be the path to these images. y will be the gender label.

In [8]:
image_reshape_size = 100
input_image_root_dir = '../Dataset/imdb_crop/' # Don't forget the ending slash

In [9]:
from keras import backend as K
K.set_image_data_format('channels_last')
batch_size = 64
inputShape = (image_reshape_size, image_reshape_size, 1)

In [10]:
df_train.head(3)

,path,id,name,dob,gender,score1,score2,pic_date,region,age
162143,58/nm0005158_rm1436916480_1970-11-20_2003.jpg,16965,Sabrina Lloyd,1970-11-20,0,4.224586,NaN,2003-01-01,"[107.93942350923744, 54.230711754618724, 187.9...",32.115649
129828,07/nm0001807_rm1897044736_1933-12-19_2007.jpg,3752,Cicely Tyson,1933-12-19,0,3.434887,NaN,2007-01-01,"[78.75376318187809, 59.19832238640856, 166.221...",73.036407
389973,79/nm0272479_rm1974701056_1975-10-22_2009.jpg,9031,Jesse Tyler Ferguson,1975-10-22,1,0.899851,NaN,2009-01-01,"[205.814, 123.83, 266.448, 184.464]",33.197122


#### Set up input image generator using flow_from_dataframe

In [11]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="training",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=batch_size,
                                            seed=1,
                                            shuffle=True)

val_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="validation",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=batch_size,
                                            seed=1,
                                            shuffle=True)

test_generator = datagen.flow_from_dataframe(dataframe=df_test, 
                                            directory=input_image_root_dir, 
                                            x_col="path", y_col=None, 
                                            class_mode=None, 
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=1,
                                            shuffle=False)

Found 176818 images belonging to 2 classes.
Found 19646 images belonging to 2 classes.
Found 10340 images.


#### (1) Run training experiments

In [ ]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            NAME = 'BN-{}-conv-{}-node-{}-dens-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))  # model name with timestamp
            print(NAME) 
            
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            checkpoint = ModelCheckpoint('weights/{}'.format(NAME), monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
            callbacks = [tensorboard, checkpoint]
            
            model = Sequential()
            
            # first layer
            model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu", input_shape=inputShape))
            model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=(3,3)))
            
            # sets up additional # of conv layers
            for _ in range(conv_layer - 1):
                layer_size *= 2
                model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
                model.add(BatchNormalization())
                model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(pool_size=(2,2)))
                model.add(Dropout(0.5))
            
            model.add(Flatten())
            
            layer_size *= 4 # to get the dense layer to be 8X of last output size
            
            # sets up # of dense layers
            for _ in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.7))
            
            # output layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            # Optional for resuming, load weights
            model.load_weights('weights/BN-5-conv-32-node-2-dens-1554706050')

            opt = Adam(lr=0.0002)
            model.compile(loss='binary_crossentropy', 
                          optimizer=opt,
                          metrics=['accuracy'])
            
            model.fit_generator(generator=train_generator,
                                steps_per_epoch=(train_generator.n // train_generator.batch_size),
                                callbacks = callbacks,
                                validation_data=val_generator,
                                validation_steps=(val_generator.n // val_generator.batch_size),
                                epochs=5,
                                use_multiprocessing=False,
                                workers=4)
            
            filepath = NAME + '.h5'
            model.save(filepath)

BN-5-conv-32-node-2-dens-1554757735
Epoch 1/5


INFO:plaidml:Analyzing Ops: 407 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 629 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 670 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 705 of 1652 operations complete


2009/2762 [====================>.........] - ETA: 29:32 - loss: 0.3630 - acc: 0.8155

INFO:plaidml:Analyzing Ops: 181 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 629 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 670 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 705 of 1652 operations complete


2761/2762 [============================>.] - ETA: 2s - loss: 0.3637 - acc: 0.8156

...

2010/2762 [====================>.........] - ETA: 29:46 - loss: 0.3608 - acc: 0.8175

INFO:plaidml:Analyzing Ops: 407 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 635 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 683 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 706 of 1652 operations complete


2761/2762 [============================>.] - ETA: 2s - loss: 0.3599 - acc: 0.8181

INFO:plaidml:Analyzing Ops: 195 of 258 operations complete


2762/2762 [==============================] - 6716s 2s/step - loss: 0.3599 - acc: 0.8181 - val_loss: 0.3781 - val_acc: 0.8075
Epoch 3/5
2762/2762 [==============================] - 6777s 2s/step - loss: 0.3605 - acc: 0.8179 - val_loss: 0.3726 - val_acc: 0.8112
Epoch 4/5
2762/2762 [==============================] - 6650s 2s/step - loss: 0.3578 - acc: 0.8197 - val_loss: 0.3778 - val_acc: 0.8097
Epoch 5/5
2111/2762 [=====================>........] - ETA: 25:40 - loss: 0.3587 - acc: 0.8184

In [24]:
model.save('CNN-final-model.h5')

#### Load model, resume training

In [ ]:
inputFile = 'BN-5-conv-32-node-2-dens-1553895953-3.h5'
model = load_model(inputFile)

In [ ]:
saveAs = 'BN-5-conv-32-node-2-dens-1553895953-4Lab'

In [ ]:
from keras.callbacks import ModelCheckpoint

NAME = saveAs
print(NAME) 

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
callbacks = [tensorboard]

opt = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

model.fit_generator(generator=train_generator,
                    steps_per_epoch=(train_generator.n // train_generator.batch_size),
                    callbacks = callbacks,
                    validation_data=val_generator,
                    validation_steps=(val_generator.n // val_generator.batch_size),
                    epochs=5,
                    use_multiprocessing=False,
                    workers=8)

filepath = NAME + '.h5'
model.save(filepath)

#### Evaluation

In [25]:
model = load_model('CNN-final-model.h5')

In [26]:
from sklearn.metrics import confusion_matrix
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=test_generator.n//test_generator.batch_size,
                            verbose=1)


10340/10340 [==============================] - 592s 57ms/step


In [27]:
y_true = df_test.gender.astype(int)
y_pred = [1 if x>=0.5 else 0 for x in pred]
cm = confusion_matrix(y_true, y_pred)

In [28]:
cm

array([[3474, 1651],
       [ 387, 4828]], dtype=int64)

In [29]:
TN = cm[0][0]
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]

Specificity

In [30]:
TN/(TN+FP)

0.6778536585365854

Precision

In [31]:
TP/(TP+FP)

0.7451767248032104

Recall

In [32]:
TP/FN

12.47545219638243

Accuracy

In [33]:
(TP+TN)/(TN+TP+FN+FP)

0.8029013539651838

#### Live Demo with image

In [ ]:
model = load_model('CNN-final-model.h5')

In [ ]:
def prepare(filepath):
    
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (100, 100))
    return new_array.reshape(-1, 100, 100, 1)

In [ ]:
prediction = model.predict(prepare('picture.jpg'))
print(prediction)

#### Live Demo with webcam